In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.cluster_analysis import prep_cluster_analysis
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from scipy import stats
from sklearn.svm import LinearSVC


In [2]:
length_per_window = 10
num_sliding_windows = int(1000/ length_per_window)

In [3]:
num_folds = 3

In [4]:
num_iterations = 5

In [ ]:
participants = ["904", "905", "906", "908", "909", "912","913", "914", "916", "917", "919",\
                "920", "921", "923", "924", "927", "928", "929", "930", "932"]

In [5]:
participants = ["105", "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]


In [6]:
X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging", length_per_window=length_per_window)
#X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials", length_per_window=length_per_window)
#X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants", length_per_window=length_per_window)


loaded


In [7]:
X[0][0].shape

(684,)

In [8]:
# Cross validation
results = {}
for i in range(num_iterations):
    for j in range(num_sliding_windows):
        #model = SVC(gamma=.001, kernel = 'rbf', C=1e-06)
        model = LinearSVC(C=1e-9, max_iter=5000)
        cv_results = cross_validate(model, X[j], y[j], cv=num_folds)
        if j in results.keys(): 
            results[j] += cv_results['test_score'].tolist()
        else:
            results[j] = cv_results['test_score'].tolist()
    print(i)

for i in range(num_sliding_windows):
    assert len(results[i]) == num_iterations * num_folds

0
1
2
3
4


In [9]:
errormean = []
stdev = []
for i in range(num_sliding_windows):
    errormean.append(np.mean(results[i]))
    stdev.append(np.std(results[i]))



In [10]:
errormean

[0.520448767945166,
 0.5305865851555384,
 0.5116190425727348,
 0.5218400797471688,
 0.5174669395404009,
 0.5291823361506062,
 0.5190060448236957,
 0.49126001944180353,
 0.49562700041500296,
 0.4883232181281038,
 0.4868417958952618,
 0.4809616965420432,
 0.48834892378783756,
 0.4825201232566697,
 0.4708239918290877,
 0.4971081414043031,
 0.49274132917722885,
 0.4942034299787392,
 0.49279901222768174,
 0.5000897448142346,
 0.49423523862333324,
 0.5000705358803198,
 0.508861787136213,
 0.4986341407385433,
 0.508881164816253,
 0.5117603955769164,
 0.527810345779935,
 0.5409427036031684,
 0.5307342661394138,
 0.5013913961750653,
 0.5058160039500094,
 0.5087015626903668,
 0.5219042595234407,
 0.4913047090406186,
 0.5132350398403978,
 0.5073549404871791,
 0.5044116424476605,
 0.48980452786352896,
 0.5000770888548473,
 0.48398838329174865,
 0.4986022195965326,
 0.4956718587599432,
 0.48689331971214606,
 0.46496954188689443,
 0.47371604729526395,
 0.48107734388649104,
 0.47958337819167823,
 0.4

In [11]:
stdev

[0.011640513868877591,
 0.03573878521286043,
 0.02266809324756967,
 0.03755877301158745,
 0.03057816614453366,
 0.019102145631150223,
 0.00516906235372587,
 0.008921600232086763,
 0.022018282100139102,
 0.006762464892177981,
 0.015507187044855335,
 0.01888080922964726,
 0.01625543202812682,
 0.02016502236373419,
 0.017836533951858056,
 0.013703211831029417,
 0.01605823317396316,
 0.022715829112292144,
 0.03071464094899449,
 0.025243964142002865,
 0.025403157392170084,
 0.019696631498304206,
 0.025275140550492865,
 0.027149618837479725,
 0.03295827068225326,
 0.02986311992913629,
 0.018895978009996203,
 0.010527494767922778,
 0.011008216393790956,
 0.0213962389996811,
 0.009823817552083393,
 0.020590273884862058,
 0.022870219603956497,
 0.030445723954765143,
 0.02177208003157512,
 0.012739762816086556,
 0.0078050255061792384,
 0.015658486447707676,
 0.026561992295509963,
 0.02127075748136071,
 0.021255964927862248,
 0.02466784345566571,
 0.017081132161239646,
 0.01598976943004282,
 0.01

In [ ]:
# T-test
accuracy_by_guessing = [0.5] * (num_iterations * num_folds)
pvalues = []
for i in range(num_sliding_windows):
    stat = stats.ttest_ind(results[i], accuracy_by_guessing)
    pvalues += [stat[1]]

In [ ]:
# Finding contiguous time cluster
valid_window = [i for i,v in enumerate(pvalues) if v >= 0.05]
print(valid_window)